In [ ]:
import pandas as pd
import glob

path = r'./iiwaridata' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
df

In [1]:
import pandas as pd
import glob
path = r'./iiwaridata' # use your path
filenames = glob.glob(path + "/*.csv")
filenames

['./iiwaridata\\node_3200.csv',
 './iiwaridata\\node_3224.csv',
 './iiwaridata\\node_3240.csv',
 './iiwaridata\\node_42787.csv',
 './iiwaridata\\node_45300.csv']

In [2]:
import pandas as pd
import glob
from datetime import datetime

start_time = datetime.now()

CHUNK_SIZE = 50000
path = r'./iiwaridata' # use your path
filenames = glob.glob(path + "/*.csv")
output_file = "./combined.csv"

for csv_file_name in filenames:
    chunk_container = pd.read_csv(csv_file_name, chunksize=CHUNK_SIZE)
    for chunk in chunk_container:
        chunk.to_csv(output_file, mode="a", index=False)
        
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:01:50.764014


In [ ]:
import pandas as pd
import time

df = pd.read_csv("./iiwaridata/node_3200.csv")
%time
df = df.astype(str)
df['timestamp'] = df['timestamp'].str.slice(2, -7)

df['timestamp'] = df['timestamp'].astype('datetime64[ns]')
df['timestamp'] = pd.Series(df['timestamp']).dt.round("S")
df.head()

In [25]:
import time
def timerfunc(func):
    """
    A timer decorator
    """
    def function_timer(*args, **kwargs):
        """
        A nested function for timing other functions
        """
        start = time.time()
        value = func(*args, **kwargs)
        end = time.time()
        runtime = end - start
        msg = "The runtime for {func} took {time} seconds to complete"
        print(msg.format(func=func.__name__,
                         time=runtime))
        return value
    return function_timer

In [ ]:
start_time = datetime.now()

def convert(data):
    data["measurement"] = ['sensordata' for t in range(len(data))]
    
    lines = [str(data["measurement"][d]) 
             + " " 
             + 'timestamp="' + str(data["timestamp"][d]) + '",' 
             + "x=" + str(data["x"][d]) + ","
             + "y=" + str(data["y"][d]) + ","
             + "z=" + str(data["z"][d]) + ","
             + "q=" + str(data["q"][d]) + ","
             + "node_id=" + str(data["node_id"][d]) for d in range(len(data))]
    return lines

def write_to_txt(text):
    thefile = open('combined.txt', 'a+')
    for item in text:
        thefile.write("%s\n" % item)
    thefile.close()


if __name__ == '__main__':
    df = pd.read_csv("./combined.csv")
    write_to_txt(convert(chunk))
        
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [ ]:
node_id,timestamp,x,y,z,q
51735,2019-03-06 15:32:26.078541+00:00,1010,100,0,140

In [ ]:
CHUNK_SIZE = 50000
path = r'./iiwaridata' # use your path
filenames = glob.glob(path + "/*.csv")
#output_file = "./combined.txt"

for csv_file_name in filenames:
    chunk_container = pd.read_csv(csv_file_name, chunksize=CHUNK_SIZE)
    
    for chunk in chunk_container:
        print(chunk)
        #chunk.to_csv(output_file, mode="a", index=False)
        #write_to_txt(convert(chunk))
        if chunk.empty:
            pass
        else:
            chunk["measurement"] = ['sensordata' for t in range(len(chunk))]
            lines = [str(chunk["measurement"][d]) 
                     + " " 
                     + 'timestamp="' + str(chunk["timestamp"][d]) + '",' 
                     + "x=" + str(chunk["x"][d]) + ","
                     + "y=" + str(chunk["y"][d]) + ","
                     + "z=" + str(chunk["z"][d]) + ","
                     + "q=" + str(chunk["q"][d]) + ","
                     + "node_id=" + str(chunk["node_id"][d]) for d in range(len(chunk))]
            
            thefile = open('combined.txt', 'a+')
            for item in lines:
                thefile.write("%s\n" % item)
            thefile.close()
            #write_to_txt(convert(chunk))

In [5]:
path = r'./iiwaridata' # use your path
filenames = glob.glob(path + "/*.csv")
sep = ";"

def check_data(data):
    # ... your tests
    return True # << True if data should be written into target file, else False

with open("/iiwaridata/combined.csv", "a+") as targetfile:
    for filename in filenames :
        with open(filename, "r") as f:
            next(f) # << only if the first line contains headers
            for line in f:
                data = line.split(sep)
                if check_data(data):
                    targetfile.write(line)

FileNotFoundError: [Errno 2] No such file or directory: '/iiwaridata/combined.csv'

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# 112m19.123s
time = np.arange(0.0, 2.0, 0.01)
items = ()

fig, ax = plt.subplots()
ax.plot(t, s)

ax.set(xlabel='time (s)', ylabel='voltage (mV)',
       title='About as simple as it gets, folks')
ax.grid()

fig.savefig("test.png")
plt.show()

In [4]:
import pandas as pd
from influxdb import InfluxDBClient
from datetime import datetime

start_time = datetime.now()

client = InfluxDBClient(host='localhost', port=8086, username='admin', password='teamfox')
client.get_list_database()

client.switch_database('iiwari_org')

tables = client.query('SELECT * FROM SensorData LIMIT 20')

test = []
for table in tables:
    #print(table)
    test.append(table)

for i in test:
    df = pd.DataFrame.from_dict(i)
    
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

df

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
#client = InfluxDBClient(url="https://eu-central-1-1.aws.cloud2.influxdata.com", token=token, org=org)  ssl=True, verify_ssl=True
#client.query('SELECT * FROM sensordata')
#http://localhost:8086/query?q=show%20databases
#from influxdb_client.client.write_api import SYNCHRONOUS

# You can generate a Token from the "Tokens Tab" in the UI
#token = "k-WR6iPY4zh2N91LGpUeqPTIAmd0HfBtdmc-YKUwJAnJSCssk4hC0C2TVIkZex2u3sQurAj_xjTGBFW38z0EUw=="
#org = "eriko.oy38@gmail.com"
#bucket = "my_bucket"

#import time

#from influxdb_client import InfluxDBClient, Point, WritePrecision, WriteOptions
#client = InfluxDBClient(url="https://eu-central-1-1.aws.cloud2.influxdata.com", token=token, org=org)  ssl=True, verify_ssl=True

In [14]:
client.switch_database('iiwari_org')

import pandas as pd

tables = client.query('SELECT * FROM sensordata LIMIT 20')

test = []
for table in tables:
    #print(table)
    test.append(table)

for i in test:
    df = pd.DataFrame.from_dict(i)
df

,time,node_id,timestamp,x,y,z
0,2020-10-25T17:32:12.062492Z,3200.0,2013-03-19 16:52:55,92.0,2593.0,0.0
1,2020-10-25T17:32:13.029819Z,3200.0,2013-03-19 18:16:28,72.0,2583.0,0.0
2,2020-10-25T17:32:13.103082Z,3200.0,2014-03-19 08:16:45,181.0,2467.0,0.0
3,2020-10-25T17:32:13.187096Z,3200.0,2014-03-19 09:36:03,-50.0,2558.0,0.0
4,2020-10-25T17:32:13.281043Z,3200.0,2014-03-19 14:22:44,120.0,2674.0,0.0
5,2020-10-25T17:32:13.344590Z,3200.0,2014-03-19 15:44:06,121.0,2656.0,0.0
6,2020-10-25T17:32:13.406945Z,3200.0,2015-03-19 06:08:39,106.0,2496.0,0.0
7,2020-10-25T17:32:13.502625Z,3200.0,2015-03-19 07:32:08,155.0,2579.0,0.0
8,2020-10-25T17:32:13.590149Z,3200.0,2015-03-19 08:55:26,140.0,2558.0,0.0
9,2020-10-25T17:32:13.653306Z,3200.0,2015-03-19 10:18:13,147.0,2601.0,0.0
